In [46]:
# https://machinelearningmastery.com/multi-step-time-series-forecasting-long-short-term-memory-networks-python/
import pandas as pd
from datetime import datetime
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import math
import matplotlib.pyplot as plt

np.random.seed(1337)

# Creating functions

In [ ]:
# transform series into train and test sets for supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names

    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    
    return agg


In [ ]:
# transform series into train and test sets for supervised learning
def prepare_data(series, n_test, n_lag, n_seq):
	# extract raw values
	raw_values = series.values
	
    # transform data to be stationary
	diff_values = raw_values
	diff_values = diff_values.reshape(len(diff_values), 1)
	
    # rescale values to -1, 1
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaled_values = scaler.fit_transform(diff_values)
	scaled_values = scaled_values.reshape(len(scaled_values), 1)
	
    # transform into supervised learning problem X, y
	supervised = series_to_supervised(scaled_values, n_lag, n_seq)
	supervised_values = supervised.values
	
    # split into train and test sets
	train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
	return scaler, train, test

In [ ]:
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
    # reshape training into [samples, timesteps, features]
    X, y = train[:, 0:n_lag], train[:, n_lag:]

    X = X.reshape(X.shape[0], 1, X.shape[1])

    # design network
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')

    model.fit(X, y, epochs=nb_epoch, batch_size=n_batch, verbose=2, shuffle=False)

    return model

In [ ]:
# make one forecast with an LSTM,
def forecast_lstm(model, X, n_batch):
    # reshape input pattern to [samples, timesteps, features]
    X = X.reshape(1, 1, len(X))
    
    # make forecast
    forecast = model.predict(X, batch_size=n_batch)
    
    # convert to array
    return [x for x in forecast[0, :]]
 
# evaluate the persistence model
def make_forecasts(model, n_batch, train, test, n_lag, n_seq):
    forecasts = list()
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
        # make forecast
        forecast = forecast_lstm(model, X, n_batch)
        # store the forecast
        forecasts.append(forecast)
    return forecasts

In [ ]:
# inverse data transform on forecasts
def inverse_transform(series, forecasts, scaler):
    inverted = list()
    for i in range(len(forecasts)):
        
        # create array from forecast
        forecast = np.array(forecasts[i])
        forecast = forecast.reshape(1, len(forecast))
        
        # invert scaling
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0, :]

        inverted.append(inv_scale)

    return inverted

# Fitting and predicting

In [ ]:
# load dataset
logreturns = 'data/final.csv'
series = pd.read_csv(logreturns, usecols=['Exchange.Date', 'logreturns'], header=0, index_col=0, squeeze=True)

# configure
n_lag = 1
n_seq = 90 #  number of periods forecast
test_share = 0.25
n_test = int(len(series) * test_share)
n_epochs = 5
n_batch = 1
n_neurons = 50

# prepare data
scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)

# fit model
model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons)

# make forecasts
forecasts = make_forecasts(model, n_batch, train, test, n_lag, n_seq)

# inverse transform forecasts and test
forecasts = inverse_transform(series, forecasts, scaler)

# Evaluating from t=1

In [68]:
# Creating a merged df with original values and predictions
original_df = pd.read_csv('data/final.csv', usecols=['logreturns', 'Close'])
train_df = original_df[:-n_test].copy()

last_train = train_df['Close'].values[-1]
price_forecasts = np.exp(np.cumsum(forecasts[0]) + math.log(last_train))

train_df['forecast'] = train_df['Close'] # temporary until actual forecasts are added

eval_df = pd.DataFrame(columns=['Close', 'logreturns', 'forecast'])
eval_df['Close'] = original_df['Close'].values[-n_test:-n_test+n_seq]
eval_df['logreturns'] = original_df['logreturns'].values[-n_test:-n_test+n_seq]
eval_df['forecast'] = price_forecasts

merged_df = train_df.append(eval_df, ignore_index=True)
merged_df['error'] = merged_df['forecast'] - merged_df['Close']

merged_df['actual_change'] = merged_df['Close'].diff(1) > 0
merged_df['forecast_change'] = merged_df['forecast'].diff(1) > 0

def correct(actual, forecast):
    if ((actual and forecast) or (not actual and not forecast)):
        return True
    return False

merged_df['correct'] = merged_df.apply(lambda x: correct(x['actual_change'], x['forecast_change']), axis=1)

merged_df.tail(n_seq)

,Close,logreturns,forecast,error,actual_change,forecast_change,correct
3423,622.77,0.002234,621.905334,-0.864666,True,True,True
3424,618.70,-0.006557,622.373718,3.673718,False,True,False
3425,617.12,-0.002557,623.175537,6.055537,False,True,False
3426,621.28,0.006718,623.923645,2.643645,True,True,True
3427,622.31,0.001656,624.617188,2.307188,True,True,True
...,...,...,...,...,...,...,...
3508,688.99,-0.007850,657.322327,-31.667673,False,True,False
3509,688.94,-0.000073,658.048645,-30.891355,False,True,False
3510,692.89,0.005717,658.999084,-33.890916,True,True,True
3511,694.34,0.002090,659.983704,-34.356296,True,True,True


In [62]:
# RMSE and MAE
def new_df(n_periods):
    df = merged_df[len(train_df):len(train_df)+n_periods]
    return df

def evaluate(n_periods):
    df = new_df(n_periods)
    mape = np.mean(np.abs((df['error']) / df['Close'])) * 100
    rmse = math.sqrt(pow(df['error'].sum(), 2) / n_periods)
    perc_correct = (len(df[df['correct'] == True]) / len(df))*100
    print(f"{n_periods}, RMSE: {round(rmse, 3)}, MAPE: {round(mape, 3)}%, Correct: {round(perc_correct, 3)}%")

# evaluate(1) # 1 day
# evaluate(3) # half a week
# evaluate(5) # week
# evaluate(21) # month
# evaluate(63) # quarter

df = new_df(63)
confusion_matrix(df['actual_change'], df['forecast_change'])

array([[ 1, 25],
       [ 0, 37]])

In [ ]:
plot_df = merged_df[-n_seq - (n_seq*2) :]
plt.figure(figsize=(10, 4))
plt.plot(plot_df['forecast'], label="forecast")
plt.plot(plot_df['Close'], label="actual")
plt.legend()

In [ ]:
eval_df['forecast'].values[0]
eval_df['forecast'].values[-1]

pred = eval_df['forecast'].values[-1] / eval_df['Close'].values[0]
act = eval_df['Close'].values[-1] / eval_df['Close'].values[0]

print(pred, act)